In [ ]:
# 安裝必要庫
!pip install emcee

# 導入庫
import numpy as np
import emcee
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from google.colab import files

"""
Dependencies:
- numpy==1.23.5
- emcee==3.1.4
- matplotlib==3.7.1
- pandas==2.0.3
- scipy==1.10.1
Data: SPARC dataset (Lelli et al. 2016), available at http://astroweb.cwru.edu/SPARC/
"""

# 上傳數據並讀取
uploaded = files.upload()
column_names = ['galaxy_id', 'rad[kpc]', 'vobs[km/s]', 'errv[km/s]', 'mass[M_sun]']
sparc_data = pd.read_csv(list(uploaded.keys())[0], sep='\s+', names=column_names, skiprows=0)
galaxy_ids = sparc_data['galaxy_id'].unique()

# 基本常數與模型定義
G = 6.674e-11         # 萬有引力常數 (m^3 kg^-1 s^-2)
M_sun = 1.989e30      # 太陽質量 (kg)
kpc_to_m = 3.085e19   # 1 kpc 轉換為 m
a0 = 1.2e-10          # MOND 加速度常數 (m/s^2)

# ----------------------------
# 修正後的四元數模型 (簡化版, 移除 0.5)
# ----------------------------
def rotation_curve_quat_simple(r_kpc, M_solar, rs_kpc, epsilon):
    """
    修正後的四元數模型（簡化版）:
      v(r) = sqrt[ (G*M_solar*M_sun*(1 - exp(-r_m/rs_m)) / r_m)
                   + (epsilon * (r_m / rs_m)) ]
    其中：
      - r_kpc 與 rs_kpc 以 kpc 為單位，實際計算轉為 m
      - M_solar 為重子質量 (單位：M_sun)
      - epsilon 為幾何修正耦合常數 (不再乘上 0.5)
    返回值最終轉為 km/s。
    """
    r_m = r_kpc * kpc_to_m
    rs_m = rs_kpc * kpc_to_m

    # 基於指數盤面的重子貢獻
    v_baryon = np.sqrt(G * M_solar * M_sun * (1 - np.exp(-r_m / rs_m)) / r_m)
    # 幾何修正項：移除「0.5」因子
    correction = epsilon * (r_m / rs_m)
    v_total = np.sqrt(v_baryon**2 + correction)
    return v_total * 1e-3  # 轉換為 km/s

# ----------------------------
# ΛCDM 模型（保持不變）
# ----------------------------
def rotation_curve_LCDM(r_kpc, M_solar, rs_kpc, M_dm, rs_dm):
    r_m = r_kpc * kpc_to_m
    v_baryon = np.sqrt(G * M_solar * M_sun / r_m)
    v_dm = np.sqrt(G * M_dm * M_sun * (1 - np.exp(-r_m / (rs_dm * kpc_to_m))) / r_m)
    return np.sqrt(v_baryon**2 + v_dm**2) * 1e-3  # km/s

def rotation_curve_LCDM_fixed(r_kpc, M_solar, rs_kpc):
    true_f_DM  = 10.0
    true_rs_DM = 15.0
    M_dm = M_solar * true_f_DM
    return rotation_curve_LCDM(r_kpc, M_solar, rs_kpc, M_dm, true_rs_DM)

# ----------------------------
# MOND 模型（含外部場效應）
# ----------------------------
def rotation_curve_MOND_ext(r_kpc, M_solar, rs_kpc, g_ext):
    r_m = r_kpc * kpc_to_m
    v_N = np.sqrt(G * M_solar * M_sun / r_m)
    a_N = v_N**2 / r_m
    mu = a_N / (a0 + g_ext)
    v_MOND = v_N * np.sqrt(0.5 + 0.5 * np.sqrt(1 + 4 / mu**2))
    return v_MOND * 1e-3  # km/s

# ----------------------------
# MCMC 擬合函數：修正後四元數模型 (簡化版，3個自由參數)
# ----------------------------
def log_likelihood_quat(theta, r, v, v_err):
    M_solar, rs_kpc, epsilon = theta
    v_model = rotation_curve_quat_simple(r, M_solar, rs_kpc, epsilon)
    return -0.5 * np.sum(((v - v_model) / v_err)**2)

def log_prior_quat(theta):
    M_solar, rs_kpc, epsilon = theta
    if 1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20 and 0 < epsilon < 1e4:
        return 0.0
    return -np.inf

def log_probability_quat(theta, r, v, v_err):
    lp = log_prior_quat(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_quat(theta, r, v, v_err)

def log_likelihood_LCDM(theta, r, v, v_err):
    M_solar, rs_kpc, M_dm, rs_dm = theta
    v_model = rotation_curve_LCDM(r, M_solar, rs_kpc, M_dm, rs_dm)
    return -0.5 * np.sum(((v - v_model) / v_err)**2)

def log_prior_LCDM(theta):
    M_solar, rs_kpc, M_dm, rs_dm = theta
    if 1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20 and 1e8 < M_dm < 1e12 and 1 < rs_dm < 50:
        return 0.0
    return -np.inf

def log_probability_LCDM(theta, r, v, v_err):
    lp = log_prior_LCDM(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_LCDM(theta, r, v, v_err)

def log_likelihood_MOND(theta, r, v, v_err):
    M_solar, rs_kpc, g_ext = theta
    v_model = rotation_curve_MOND_ext(r, M_solar, rs_kpc, g_ext)
    return -0.5 * np.sum(((v - v_model) / v_err)**2)

def log_prior_MOND(theta):
    M_solar, rs_kpc, g_ext = theta
    if 1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20 and 0 < g_ext < 1e-9:
        return 0.0
    return -np.inf

def log_probability_MOND(theta, r, v, v_err):
    lp = log_prior_MOND(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_MOND(theta, r, v, v_err)

def get_confidence_intervals(samples, percentiles=[16, 84]):
    return [np.percentile(samples[:, i], percentiles) for i in range(samples.shape[1])]

# ----------------------------
# 逐星系擬合與比較
# ----------------------------
results_list = []
nwalkers = 32

for galaxy in galaxy_ids:
    print(f"Processing {galaxy}...")
    galaxy_data = sparc_data[sparc_data['galaxy_id'] == galaxy].dropna()
    galaxy_data = galaxy_data[(galaxy_data['vobs[km/s]'] > 0) & (galaxy_data['errv[km/s]'] >= 0)]
    if len(galaxy_data) < 4:
        print(f"Skipping {galaxy} due to insufficient data points.")
        continue

    r_kpc_data = galaxy_data['rad[kpc]'].values
    v_obs = galaxy_data['vobs[km/s]'].values
    v_err = galaxy_data['errv[km/s]'].values.clip(min=0.1)
    mass_sun_val = galaxy_data['mass[M_sun]'].values[0]

    # 修正後的四元數模型 MCMC (3自由參數)
    ndim_quat = 3
    p0_quat = np.array([mass_sun_val, 3.0, 0.05]) * (1 + 0.1 * np.random.randn(nwalkers, ndim_quat))
    sampler_quat = emcee.EnsembleSampler(nwalkers, ndim_quat, log_probability_quat, args=(r_kpc_data, v_obs, v_err))
    sampler_quat.run_mcmc(p0_quat, 2000, progress=True)
    samples_quat = sampler_quat.get_chain(discard=500, flat=True)
    M_solar_quat, rs_kpc_quat, epsilon_quat = np.median(samples_quat, axis=0)
    ci_quat = get_confidence_intervals(samples_quat)
    v_quat_fit = rotation_curve_quat_simple(r_kpc_data, M_solar_quat, rs_kpc_quat, epsilon_quat)
    chi2_quat = np.sum(((v_obs - v_quat_fit) / v_err)**2)
    dof_quat = len(v_obs) - ndim_quat
    red_chi2_quat = chi2_quat / dof_quat if dof_quat > 0 else np.nan
    AIC_quat = chi2_quat + 2 * ndim_quat
    BIC_quat = chi2_quat + ndim_quat * np.log(len(v_obs))

    # ΛCDM 模型 MCMC
    ndim_LCDM = 4
    p0_LCDM = np.array([mass_sun_val, 3.0, mass_sun_val * 5, 15.0]) * (1 + 0.1 * np.random.randn(nwalkers, ndim_LCDM))
    sampler_LCDM = emcee.EnsembleSampler(nwalkers, ndim_LCDM, log_probability_LCDM, args=(r_kpc_data, v_obs, v_err))
    sampler_LCDM.run_mcmc(p0_LCDM, 2000, progress=True)
    samples_LCDM = sampler_LCDM.get_chain(discard=500, flat=True)
    M_solar_LCDM, rs_kpc_LCDM, M_dm_LCDM, rs_dm_LCDM = np.median(samples_LCDM, axis=0)
    ci_LCDM = get_confidence_intervals(samples_LCDM)
    v_LCDM_fit = rotation_curve_LCDM(r_kpc_data, M_solar_LCDM, rs_kpc_LCDM, M_dm_LCDM, rs_dm_LCDM)
    chi2_LCDM = np.sum(((v_obs - v_LCDM_fit) / v_err)**2)
    dof_LCDM = len(v_obs) - ndim_LCDM
    red_chi2_LCDM = chi2_LCDM / dof_LCDM if dof_LCDM > 0 else np.nan
    AIC_LCDM = chi2_LCDM + 2 * ndim_LCDM
    BIC_LCDM = chi2_LCDM + ndim_LCDM * np.log(len(v_obs))

    # MOND 模型 MCMC
    ndim_MOND = 3
    p0_MOND = np.array([mass_sun_val, 3.0, 1e-10]) * (1 + 0.1 * np.random.randn(nwalkers, ndim_MOND))
    sampler_MOND = emcee.EnsembleSampler(nwalkers, ndim_MOND, log_probability_MOND, args=(r_kpc_data, v_obs, v_err))
    sampler_MOND.run_mcmc(p0_MOND, 2000, progress=True)
    samples_MOND = sampler_MOND.get_chain(discard=500, flat=True)
    M_solar_MOND, rs_kpc_MOND, g_ext_MOND = np.median(samples_MOND, axis=0)
    ci_MOND = get_confidence_intervals(samples_MOND)
    v_MOND_fit = rotation_curve_MOND_ext(r_kpc_data, M_solar_MOND, rs_kpc_MOND, g_ext_MOND)
    chi2_MOND = np.sum(((v_obs - v_MOND_fit) / v_err)**2)
    dof_MOND = len(v_obs) - ndim_MOND
    red_chi2_MOND = chi2_MOND / dof_MOND if dof_MOND > 0 else np.nan
    AIC_MOND = chi2_MOND + 2 * ndim_MOND
    BIC_MOND = chi2_MOND + ndim_MOND * np.log(len(v_obs))

    # 根據 AIC 選擇最佳模型
    aic_dict = {'Quaternionic': AIC_quat, 'LCDM': AIC_LCDM, 'MOND': AIC_MOND}
    best_model = min(aic_dict, key=aic_dict.get) if all(~np.isnan(list(aic_dict.values()))) else "N/A"

    results_list.append({
        'Galaxy': galaxy,
        'chi2_Quat': chi2_quat, 'red_chi2_Quat': red_chi2_quat, 'AIC_Quat': AIC_quat, 'BIC_Quat': BIC_quat,
        'M_solar_Quat': M_solar_quat, 'rs_kpc_Quat': rs_kpc_quat, 'epsilon_Quat': epsilon_quat,
        'M_solar_Quat_68_low': ci_quat[0][0], 'M_solar_Quat_68_high': ci_quat[0][1],
        'rs_kpc_Quat_68_low': ci_quat[1][0], 'rs_kpc_Quat_68_high': ci_quat[1][1],
        'epsilon_Quat_68_low': ci_quat[2][0], 'epsilon_Quat_68_high': ci_quat[2][1],
        'chi2_LCDM': chi2_LCDM, 'red_chi2_LCDM': red_chi2_LCDM, 'AIC_LCDM': AIC_LCDM, 'BIC_LCDM': BIC_LCDM,
        'M_solar_LCDM': M_solar_LCDM, 'rs_kpc_LCDM': rs_kpc_LCDM, 'M_dm_LCDM': M_dm_LCDM, 'rs_dm_LCDM': rs_dm_LCDM,
        'chi2_MOND': chi2_MOND, 'red_chi2_MOND': red_chi2_MOND, 'AIC_MOND': AIC_MOND, 'BIC_MOND': BIC_MOND,
        'M_solar_MOND': M_solar_MOND, 'rs_kpc_MOND': rs_kpc_MOND, 'g_ext_MOND': g_ext_MOND,
        'Best_Model': best_model
    })

    # 繪圖
    plt.figure(figsize=(10, 6))
    plt.errorbar(r_kpc_data, v_obs, yerr=v_err, fmt='o', label=f'{galaxy} Data', alpha=0.5)
    plt.plot(r_kpc_data, v_quat_fit, label=f'Quaternionic (M={M_solar_quat:.2e}, ε={epsilon_quat:.4f})')
    plt.plot(r_kpc_data, v_LCDM_fit, label=f'ΛCDM (M={M_solar_LCDM:.2e}, M_dm={M_dm_LCDM:.2e})')
    plt.plot(r_kpc_data, v_MOND_fit, label=f'MOND (M={M_solar_MOND:.2e}, g_ext={g_ext_MOND:.2e})')
    plt.xscale('log')
    plt.xlabel('Radius (kpc)')
    plt.ylabel('Rotation Velocity (km/s)')
    v_min = min(v_obs.min() - v_err.max(), v_quat_fit.min(), v_LCDM_fit.min(), v_MOND_fit.min()) * 0.9
    v_max = max(v_obs.max() + v_err.max(), v_quat_fit.max(), v_LCDM_fit.max(), v_MOND_fit.max()) * 1.1
    plt.ylim(v_min, v_max)
    plt.legend()
    plt.title(f'Rotation Curve Fits for {galaxy}')
    plt.grid(True, which="both", ls="--")
    plt.show()

# 結果表格與總統計
df_results = pd.DataFrame(results_list)
summary_columns = ['Galaxy', 'red_chi2_Quat', 'AIC_Quat', 'red_chi2_LCDM', 'AIC_LCDM', 'red_chi2_MOND', 'AIC_MOND', 'Best_Model']
print(df_results[summary_columns].to_string(index=False))
df_results.to_csv("sparc_rotation_curve_comparison_all.csv", index=False)
print("已將結果儲存為 'sparc_rotation_curve_comparison_all.csv'")

total_chi2_quat = df_results['chi2_Quat'].sum()
total_dof_quat = sum(len(sparc_data[sparc_data['galaxy_id'] == g]) - ndim_quat for g in galaxy_ids if g in df_results['Galaxy'].values)
total_red_chi2_quat = total_chi2_quat / total_dof_quat
total_chi2_LCDM = df_results['chi2_LCDM'].sum()
total_dof_LCDM = sum(len(sparc_data[sparc_data['galaxy_id'] == g]) - ndim_LCDM for g in galaxy_ids if g in df_results['Galaxy'].values)
total_red_chi2_LCDM = total_chi2_LCDM / total_dof_LCDM
total_chi2_MOND = df_results['chi2_MOND'].sum()
total_dof_MOND = sum(len(sparc_data[sparc_data['galaxy_id'] == g]) - ndim_MOND for g in galaxy_ids if g in df_results['Galaxy'].values)
total_red_chi2_MOND = total_chi2_MOND / total_dof_MOND

print(f"\nTotal reduced chi2 (Quaternionic): {total_red_chi2_quat:.2f}")
print(f"Total reduced chi2 (LCDM): {total_red_chi2_LCDM:.2f}")
print(f"Total reduced chi2 (MOND): {total_red_chi2_MOND:.2f}")

best_model_counts = df_results['Best_Model'].value_counts()
print("\n最佳模型統計：")
print(best_model_counts)

outliers = df_results[df_results['chi2_Quat'] > 10000]
print("\nOutliers with chi2_Quat > 10000:")
print(outliers[['Galaxy', 'chi2_Quat', 'AIC_Quat']].to_string(index=False))